In [1]:
#programa utilizado para o exemplo de grafos para aplicação do PageRank

In [2]:
from pyspark.sql import SparkSession #importa a biblioteca que cria a seção do spark

In [3]:
#inicia a seção para a utilização do spark
spark = SparkSession.builder.appName("pageRankGrafos").getOrCreate() #cria a seção caso não exista ou obtém a já criada

Criando o nosso grafo

In [5]:
#criando os nós para o nosso grafo
nos = spark.createDataFrame([
    ("A", "ANA"  ,350 ),
    ("B", "BERNARDO"  ,360 ),
    ("C", "CLARA" ,195 ),
    ("D", "DANIEL",90),
    ("E", "ERIC"  ,90),
    ("F", "FERNANDA" ,215 ),
    ("G", "GUSTAVO",30 ),
    ("H", "HENRIQUE" ,25 ),
    ("I", "IOLANDA"  ,25 ),
    ("J", "JENNIFER"   ,20 )
], ["id", "nome", "total_segundos"])

In [6]:
nos.show()

+---+--------+--------------+
 id| nome|total_segundos|
+---+--------+--------------+
 A| ANA| 350|
 B|BERNARDO| 360|
 C| CLARA| 195|
 D| DANIEL| 90|
 E| ERIC| 90|
 F|FERNANDA| 215|
 G| GUSTAVO| 30|
 H|HENRIQUE| 25|
 I| IOLANDA| 25|
 J|JENNIFER| 20|
+---+--------+--------------+

In [7]:
relacionamentos=spark.createDataFrame([
    ("A", "B", 60),
    ("B", "A", 50),
    ("A", "C", 50),
    ("C", "A", 100),
    ("A", "D", 90),
    ("C", "I", 25),
    ("C", "J", 20),
    ("B", "F", 50),
    ("F", "B", 110),
    ("F", "G", 30),
    ("F", "H", 25),
    ("B", "E", 90)
],["src","dst","duracao_chamada"])

In [8]:
relacionamentos.show() # a fonte (src) indica quem ligou e (dst) indica quem recebeu a ligação

+---+---+---------------+
src|dst|duracao_chamada|
+---+---+---------------+
 A| B| 60|
 B| A| 50|
 A| C| 50|
 C| A| 100|
 A| D| 90|
 C| I| 25|
 C| J| 20|
 B| F| 50|
 F| B| 110|
 F| G| 30|
 F| H| 25|
 B| E| 90|
+---+---+---------------+

Definindo e criando o grafo

In [10]:
#importando as funções para utilizar os grafos
from pyspark.sql.types import *
from graphframes import *  #contém os métodos para serem utilizados no processamento através dos grafos
#obs: a biblioteca graphframes deve ser adicionada ao databricks, pois não é nativa. Para isso acesse New->"Library"-> "Source"-> "Maven Coordinate"-> pesquise por "graphframes" e adicione a biblioteca para cada cluster

In [11]:
#controi o grafo a partir dos dataframes
grafo = GraphFrame(nos,relacionamentos)


Explorando o nosso grafo

In [13]:
#encontrando chamadas com duração total maior do que 150 min
from pyspark.sql.functions import col

grafo.vertices\
.filter("total_segundos > 150")\
.sort(col("total_segundos").desc())\
.show()

+---+--------+--------------+
 id| nome|total_segundos|
+---+--------+--------------+
 B|BERNARDO| 360|
 A| ANA| 350|
 F|FERNANDA| 215|
 C| CLARA| 195|
+---+--------+--------------+

In [14]:
#encontrando as estatísticas para o total de segundos existentes no grafo (para cada um dos nós) 
grafo.vertices\
.describe(['total_segundos'])\
.show()

+-------+-----------------+
summary| total_segundos|
+-------+-----------------+
 count| 10|
 mean| 140.0|
 stddev|132.9578045011942|
 min| 20|
 max| 360|
+-------+-----------------+

In [15]:
#encontrando as estatísticas para cada uma das ligações (relacionamentos)
grafo.edges\
.describe(['duracao_chamada'])\
.show()

+-------+------------------+
summary| duracao_chamada|
+-------+------------------+
 count| 12|
 mean|58.333333333333336|
 stddev| 31.79003083682148|
 min| 20|
 max| 110|
+-------+------------------+

In [16]:
#mostrando a quantidade de caminhos diretos (chegando) ->  ligações recebidas
display(grafo.inDegrees)

id,inDegree
F,1
E,1
B,2
D,1
C,1
J,1
A,2
G,1
I,1
H,1


In [17]:
#mostrando a quantidade de caminhos inversos (saindo) -> ligações efetuadas
display(grafo.outDegrees)

id,outDegree
F,3
B,3
C,3
A,3


In [18]:
#qual é o nó mais "importante" (tem mais caminhos que levam até ele) -> quem mais recebeu ligações
total_degree = grafo.degrees
in_degree = grafo.inDegrees
out_degree = grafo.outDegrees


In [19]:
total_degree.show()

+---+------+
 id|degree|
+---+------+
 F| 4|
 E| 1|
 B| 5|
 D| 1|
 C| 4|
 J| 1|
 A| 5|
 G| 1|
 I| 1|
 H| 1|
+---+------+

In [20]:
#realizando a união dos dois dataframes (in e out degree)
#podem existir nós que não ligaram ou receberam ligação, assim é necessário preencher o Nan
#fica mais interessante mostrar em ordem decrescente os dados
total_degree.join(in_degree, "id", how="left")\
.join(out_degree, "id", how="left")\
.fillna(0)\
.sort("inDegree", ascending=False)\
.show()

+---+------+--------+---------+
 id|degree|inDegree|outDegree|
+---+------+--------+---------+
 B| 5| 2| 3|
 A| 5| 2| 3|
 F| 4| 1| 3|
 E| 1| 1| 0|
 J| 1| 1| 0|
 D| 1| 1| 0|
 C| 4| 1| 3|
 G| 1| 1| 0|
 I| 1| 1| 0|
 H| 1| 1| 0|
+---+------+--------+---------+

Aplicando o algoritmo PageRank

In [22]:
pageRank = grafo.pageRank(resetProbability=0.15, tol=0.001) # resetProbability= probabilidade de sair de uma página e visitar outra sem link direto (garante que todas possam ser visitadas)-> entre 0 e 1
#tol=tolerância->indica o critério de parada(se não melhorou o anterior em tol-valor)

In [23]:
#page rank gera ou outro grafo 
#indicando quais são os nós mais "importantes" em nosso grafo 
pageRank.vertices.sort(['pagerank'],ascending=False).show()

+---+--------+--------------+------------------+
 id| nome|total_segundos| pagerank|
+---+--------+--------------+------------------+
 B|BERNARDO| 360|1.2519236648772267|
 A| ANA| 350|1.2519236648772267|
 F|FERNANDA| 215|0.9759477302972548|
 D| DANIEL| 90|0.9759477302972548|
 C| CLARA| 195|0.9759477302972548|
 E| ERIC| 90|0.9759477302972548|
 H|HENRIQUE| 25|0.8980904372641323|
 G| GUSTAVO| 30|0.8980904372641323|
 I| IOLANDA| 25|0.8980904372641323|
 J|JENNIFER| 20|0.8980904372641323|
+---+--------+--------------+------------------+

In [24]:
#identificando os pesos existentes entre cada conexão
pageRank.edges.show() #page rank não leva em consideração os pesos, apenas os tipos de relacionamentos. Assim, realiza a normalização dos pesos encontrados.

+---+---+---------------+------------------+
src|dst|duracao_chamada| weight|
+---+---+---------------+------------------+
 C| J| 20|0.3333333333333333|
 A| B| 60|0.3333333333333333|
 B| F| 50|0.3333333333333333|
 C| I| 25|0.3333333333333333|
 F| H| 25|0.3333333333333333|
 C| A| 100|0.3333333333333333|
 B| A| 50|0.3333333333333333|
 F| G| 30|0.3333333333333333|
 A| D| 90|0.3333333333333333|
 F| B| 110|0.3333333333333333|
 B| E| 90|0.3333333333333333|
 A| C| 50|0.3333333333333333|
+---+---+---------------+------------------+